## Homework 7

## Arthur Setiawan 11-09-2023

In [1]:
import pandas as pd
import altair as alt
import geopandas as gpd
import osmnx as ox
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#Seattle Unreinforced Masonry Buildings Open Dataset
data = pd.read_csv('sea_masonry.csv')

# Extract the longs and lats w/ regex
data[['Longitude', 'Latitude']] = data['Long/Lat'].str.extract(r'POINT \(([-\d.]+) ([-\d.]+)\)')

# Convert the extracted values to numeric data types
data['Longitude'] = pd.to_numeric(data['Longitude'])
data['Latitude'] = pd.to_numeric(data['Latitude'])

data = data.drop(columns=['Long/Lat'])

data.head()

,Preliminary Risk Category,Neighborhood,Address,City,State,Zip Code,Year Built,No. Stories,Retrofit Level,Building Use,Estimated Number of Occupants,Confirmation Source,Longitude,Latitude
0,Medium Risk,Alki/Admiral,1321 Harbor Ave SW,Seattle,WA,98116,1915,1,No visible retrofit,Commercial,11-100,Photos from Previous Surveys,-122.382732,47.591400
1,Medium Risk,Alki/Admiral,2124 California Ave SW,Seattle,WA,98116,1928,3,No visible retrofit,Residential,11-100,Google Street View,-122.386073,47.584527
2,Medium Risk,Alki/Admiral,2306 42nd Ave SW,Seattle,WA,98116,1909,2,Visible retrofit,Government,11-100,Photos (Rosettes) from Previous Surveys,-122.384726,47.582694
3,Medium Risk,Alki/Admiral,4029 Beach Dr SW,Seattle,WA,98116,1928,1,No visible retrofit,Commercial/Residential,1-10,Field Visit,-122.409789,47.567839
4,Medium Risk,Ballard,5300 Ballard Ave NW,Seattle,WA,98107,1902,2,No visible retrofit,Residential,11-100,Field Visit,-122.383050,47.666624


In [3]:
#Taking Subset of Data that we're interested in, e.g. we only want too look at schools, government, industrial, and office buildings
data_interest_m1 = data.loc[(data['Building Use'] == 'Schools') | 
                         (data['Building Use'] == 'Government') |
                         (data['Building Use'] == 'Industrial') |
                         (data['Building Use'] == 'Office')]

data_interest_sample_m1 = data_interest_m1.sample(n=20)

data_interest_sample_m1.head()

,Preliminary Risk Category,Neighborhood,Address,City,State,Zip Code,Year Built,No. Stories,Retrofit Level,Building Use,Estimated Number of Occupants,Confirmation Source,Longitude,Latitude
116,Medium Risk,Belltown,87 Wall St,Seattle,WA,98121,1912,2,No visible retrofit,Office,11-100,Google Street View,-122.348996,47.614115
1027,Critical Risk,Univ Dist,4105 Memorial Way NE,Seattle,WA,98195,1902,4,Permitted Retrofit,Schools,101+,University of Washington Daily--Online,-122.309751,47.658920
373,Critical Risk,Central Area/Squire Park,1424 15th Ave,Seattle,WA,98122,1924,2,Permitted Retrofit,Schools,101+,Photos from Previous Surveys,-122.311866,47.613170
624,Medium Risk,Georgetown,5421 1st Ave S,Seattle,WA,98108,1941,1,No visible retrofit,Industrial,1-10,Permit Drawing Review,-122.334821,47.553657
982,Critical Risk,Rainier Beach,9709 60th Ave S,Seattle,WA,98118,1909,3,Permitted Retrofit,Schools,101+,Permit Drawing Review,-122.258843,47.514670


In [4]:
#Taking Subset of Data that we're interested in, e.g. we look at commercial or residential buildings
data_interest_m2 = data.loc[(data['Building Use'] == 'Commercial') | 
                         (data['Building Use'] == 'Residential')]

data_interest_sample_m2 = data_interest_m2.sample(n=20)

data_interest_sample_m2.head()

,Preliminary Risk Category,Neighborhood,Address,City,State,Zip Code,Year Built,No. Stories,Retrofit Level,Building Use,Estimated Number of Occupants,Confirmation Source,Longitude,Latitude
554,Medium Risk,Fauntleroy/Seaview,6045 California Ave SW,Seattle,WA,98136,1926,1,No visible retrofit,Commercial,1-10,Field Visit,-122.387468,47.547117
976,Medium Risk,Queen Anne,731 Westlake Ave N,Seattle,WA,98109,1921,2,No visible retrofit,Commercial,101+,Field Visit,-122.339396,47.626332
915,Medium Risk,Queen Anne,916 Queen Anne Ave N,Seattle,WA,98109,1909,5,Substantial Alteration,Residential,11-100,Permit Drawing Review,-122.356310,47.627850
466,Medium Risk,Downtown,110 Cherry St,Seattle,WA,98104,1903,3,No visible retrofit,Commercial,11-100,Google Street View,-122.334088,47.602846
143,Medium Risk,Belltown,1901 3rd Ave,Seattle,WA,98101,1916,2,No visible retrofit,Commercial,101+,Google Street View,-122.340147,47.611598


In [9]:
#Initialize Dual Map Showing Schools, Office, Government, and Industrial Buildings on 1 side (left) and
#commercial and residential buildings on the right side of the map.
m = folium.Map(location=[47.61, -122.3], 
               tiles='cartodbpositron',
               zoom_start=11.5, control_scale=True)


#Specifics for Map 1

for idx, row in data_interest_sample_m1.iterrows():
    # Get lat and lon of points
    lon = row['Longitude']
    lat = row['Latitude']
    
    # Tooltip = address of location
    name = row['Neighborhood'] + ', ' + row['Building Use'] + ', ' + row['Address'] + ', '+ row['City'] + ' ' + row['State'] + ', ' + str(row['Zip Code'])
    
    # Change Icon based on Building Use
    if row['Building Use'] == 'Schools':
        icon = folium.Icon(icon = 'school', color = 'cadetblue', prefix = 'fa')
        building_type = 'Schools'
        
    elif row['Building Use'] == 'Office':
        icon = folium.Icon(icon = 'briefcase', color = 'purple', prefix = 'fa')
        building_type = 'Office'
        
    elif row['Building Use'] == 'Government':
        icon = folium.Icon(icon = 'building-columns', color ='lightgray', prefix = 'fa')
        building_type = 'Government'
        
    elif row['Building Use'] == 'Industrial':
        icon = folium.Icon(icon = 'industry', color ='darkgreen', prefix = 'fa')
        building_type = 'Industrial'
        
    # Adding Filter for preliminary risk category
    if row['Preliminary Risk Category'] == 'Medium Risk':
        risk_type = 'Medium Risk'
    elif row['Preliminary Risk Category'] == 'High Risk':
        risk_type = 'High Risk'
    elif row['Preliminary Risk Category'] == 'Critical Risk':
        risk_type = 'Critical Risk'

    # Add marker to the map
    folium.Marker(location=[lat, lon],
                  tooltip=name,
                  icon=icon,
                  tags=[risk_type, building_type]
    ).add_to(m)
    
#Define category lists
risk_categories = ['Medium Risk','High Risk', 'Critical Risk']
building_categories = ['Schools', 'Office', 'Government', 'Industry']
    
#Add 2 Filters, one fo Risk, one for Building Types for the map
plugins.TagFilterButton(risk_categories,position="topright").add_to(m)
plugins.TagFilterButton(building_categories, position="topright").add_to(m)

#Fullscreen button for Map 1
folium.plugins.Fullscreen(
    position="topright",
    title="Fullscreen Mode",
    title_cancel="Exit Fullscreen",
    force_separate_button=True,
).add_to(m)
    
heat_data = [[row['Latitude'],row['Longitude']] for index, row in data_interest_sample_m2.iterrows()]

HeatMap(heat_data).add_to(m)

#Define category lists
risk_categories = ['Medium Risk','High Risk', 'Critical Risk']
building_categories = ['Commercial', 'Residential']
    
m

In [10]:
#Save map as html
outfp = "seattle_masonry.html"
m.save(outfp)